In [1]:
import os, re, json, random, math
from typing import List

import torch
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# ===== 기본 설정 =====
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ 모델을 {device}에서 실행합니다.")

# 💡 sentence-transformers 전용 모델로 변경
MODEL_NAME = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
DATA_DIR = "/home/ds4_sia_nolb/llama_index_json/2024"
OUTPUT_DIR = "/home/ds4_sia_nolb/code/confirmed/summarized_llama_articles"
OUTPUT_FILENAME = "summarized_articles_ballon.json"

# ===== 모델 로드 =====
model = SentenceTransformer(MODEL_NAME, device=device)
print(f"✅ KoBERT 기반 모델 로드 완료: {MODEL_NAME}")

max_model_input_length = 512
print(f"모델 최대 입력 길이: {max_model_input_length}")

# ===== 전처리 =====
NOISE_PATTERNS = [
    r"\(영상[^\)]*\)",
    r"<저작권자\(c\).*?>",
    r"무단전재\s*및\s*재배포\s*금지",
    r"재배포\s*금지",
    r"재판매\s*및\s*DB\s*금지",
    r"재판매\s*금지",
    r"DB\s*금지",
    r"제보는\s*카카오톡.*",
    r"구독중|구독 해지|구독|이전\s*다음",
    r"연합뉴스\s*TV|연합\s*뉴스",
    r"뉴스레터\s*구독.*",
    r"ⓒ\s*연합뉴스",
    r"송고\s*시간[:：]?\s*\d{2}:\d{2}",
    r"기사\s*입력\s*[:：]?.*",
    r"기자\s*=\s*",
    r"\(서울=.*?\)\s*",
    r"\([^)]+연합뉴스\)\s*",
    r"\([^)]+=.*?\)\s*",
    r"\[.*?\]",              # 대괄호 전체 제거
    r"이미지\s*확대",
]

VIDEO_CLOSE_SPAN_RE = re.compile(
    r'(?:(?<=^)|(?<=[\.\?\!。？！…\n,，;:]))\s*.*?영상\s*닫기\s*',
    re.IGNORECASE
)

EMAIL_RE = re.compile(r"[A-Za-z0-9._%+\-]+@[A-Za-z0-9.\-]+\.[A-Za-z]{2,}")

# 이름 패턴(한글/영문)
NAME_PATTERN = r"(?:[가-힣][가-힣·]{1,4}(?:\s[가-힣·]{2,4})?|[A-Za-z][A-Za-z .'\-]{0,30}[A-Za-z])"
# 기자/직함이 붙은 경우
REPORTER_WITH_SUFFIX_RE = re.compile(
    rf"({NAME_PATTERN})\s*(?:기자|특파원|통신원|논설위원|평론가)",
    re.IGNORECASE
)
# 이메일 앞에 이름이 오는 경우
REPORTER_BEFORE_EMAIL_RE = re.compile(
    rf"({NAME_PATTERN})(?=\s*[<\(\[]?\s*[A-Za-z0-9._%+\-]+@[A-Za-z0-9.\-]+\.[A-Za-z]{{2,}}\s*[>\)\]]?)"
)

# ===== 전처리 함수 =====
def preprocess_text(text: str):
    if pd.isna(text) or not isinstance(text, str):
        return "", set()

    t = text

    # 0) "… 영상 닫기" 절 제거
    t = VIDEO_CLOSE_SPAN_RE.sub(" ", t)

    # 1) 노이즈 패턴 제거
    for pat in NOISE_PATTERNS:
        t = re.sub(pat, " ", t, flags=re.IGNORECASE | re.DOTALL)

    # 2) 이메일 제거
    t = EMAIL_RE.sub(" ", t)

    # 3) 기자 이름 추출(두 계열 모두)
    names_with_suffix = {m.group(1).strip() for m in REPORTER_WITH_SUFFIX_RE.finditer(text)}
    names_before_email = {m.group(1).strip() for m in REPORTER_BEFORE_EMAIL_RE.finditer(text)}
    reporter_names = (names_with_suffix | names_before_email)

    # 4) 공백 정리
    t = re.sub(r"[\u200b\u200e\u200f\ufeff]", "", t)
    t = re.sub(r"\s+", " ", t).strip(" ,，;:")

    return t, reporter_names

# ===== 요약 후 summary 후처리 =====
def clean_summary(summary: str, reporter_names: set) -> str:
    if not isinstance(summary, str):
        return ""
    cleaned = summary

    for name in reporter_names:
        # 이름 경계
        name_boundary = rf"(?<![가-힣A-Za-z·]){re.escape(name)}(?![가-힣A-Za-z·])"
        # 이름+직함 제거
        cleaned = re.sub(
            rf"{name_boundary}\s*(?:기자|특파원|통신원|논설위원|평론가)",
            "",
            cleaned,
            flags=re.IGNORECASE
        )
        # 이름 단독 제거
        cleaned = re.sub(name_boundary, "", cleaned)

    # 이메일 제거
    cleaned = EMAIL_RE.sub("", cleaned)

    # 공백/구두점 정리
    cleaned = re.sub(r"\s+", " ", cleaned).strip(" ,，;:")
    return cleaned

def split_sentences_ko(text: str) -> List[str]:
    try:
        import kss
        return [s.strip() for s in kss.split_sentences(text) if s.strip()]
    except Exception:
        sents = re.split(r'(?<=[\.!?])\s+(?=[“"(\[]?[가-힣A-Z0-9])', text)
        return [s.strip() for s in sents if s.strip()]

# ===== 추출 요약기 (Sentence-Transformers 활용) =====
def summarize_extractive(clean_text: str, num_sentences: int = 3) -> str:
    if not clean_text:
        return ""
    sentences = split_sentences_ko(clean_text)
    if len(sentences) <= num_sentences:
        return " ".join(sentences)

    sentence_embeddings = model.encode(sentences, convert_to_tensor=True)
    document_embedding = torch.mean(sentence_embeddings, dim=0).unsqueeze(0)
    similarities = cosine_similarity(
        sentence_embeddings.cpu().numpy(),
        document_embedding.cpu().numpy()
    ).flatten()

    top_sentence_indices = similarities.argsort()[-num_sentences:][::-1]
    top_sentence_indices.sort()
    summarized_sentences = [sentences[i] for i in top_sentence_indices]
    return " ".join(summarized_sentences)

# ===== 실행부 =====
if __name__ == "__main__":
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    all_data = []
    for filename in os.listdir(DATA_DIR):
        if filename.endswith(".json"):
            path = os.path.join(DATA_DIR, filename)
            try:
                with open(path, "r", encoding="utf-8") as f:
                    data = json.load(f)
                if isinstance(data, list):
                    all_data.extend(data)
                else:
                    all_data.append(data)
            except Exception as e:
                print(f"⚠️ 파일 로드 실패: {path} -> {e}")

    if not all_data:
        print("❌ JSON 데이터가 없습니다.")
        raise SystemExit(0)

    print(f"📄 총 기사 수: {len(all_data)}건")

    for i, item in enumerate(tqdm(all_data, desc="기사 요약 중")):
        text = item.get("text", "")
        try:
            clean_text, reporter_names = preprocess_text(text)
            summary = summarize_extractive(clean_text, num_sentences=4)
            summary = clean_summary(summary, reporter_names)
        except Exception as e:
            summary = ""
            print(f"⚠️ 요약 실패(index={i}): {e}")
        item["summary"] = summary

    output_path = os.path.join(OUTPUT_DIR, OUTPUT_FILENAME)
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=4)

    print(f"✅ {len(all_data)}건의 요약 결과 저장 완료: {output_path}")


/home/ds4_sia_nolb/miniconda3/envs/vllm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ 모델을 cuda에서 실행합니다.
✅ KoBERT 기반 모델 로드 완료: snunlp/KR-SBERT-V40K-klueNLI-augSTS
모델 최대 입력 길이: 512
📄 총 기사 수: 10470건


기사 요약 중: 100%|██████████| 10470/10470 [06:26<00:00, 27.09it/s]


✅ 10470건의 요약 결과 저장 완료: /home/ds4_sia_nolb/code/confirmed/summarized_llama_articles/summarized_articles_ballon.json
